In [52]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
862,application_1761923966900_0874,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
847,application_1761923966900_0859,pyspark,idle,Link,Link,None,
848,application_1761923966900_0860,pyspark,idle,Link,Link,None,
849,application_1761923966900_0861,pyspark,idle,Link,Link,None,
850,application_1761923966900_0862,pyspark,idle,Link,Link,None,
851,application_1761923966900_0863,pyspark,idle,Link,Link,None,
856,application_1761923966900_0868,pyspark,idle,Link,Link,None,
857,application_1761923966900_0869,pyspark,idle,Link,Link,None,
858,application_1761923966900_0870,pyspark,idle,Link,Link,None,
859,application_1761923966900_0871,pyspark,idle,Link,Link,None,
861,application_1761923966900_0873,pyspark,busy,Link,Link,None,


In [54]:
from sedona.spark import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from sedona.spark import SedonaContext
from sedona.register.geo_registrator import SedonaRegistrator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import *
from sedona.sql.types import GeometryType
from sedona.sql import ST_Point, ST_Distance
from pyspark.sql.functions import min as sql_min
import time

start_time = time.time()

# Create spark Session
spark = SparkSession.builder \
    .appName("GeoJSON read") \
    .getOrCreate()
# Φορτώνουμε όλα τα εγκλήματα που διαπράχθηκαν την διετία 2020-2021 και τα μετατρέπουμε σε σημεία ST_Point, έπειτα καταχωρούμε τα εγκλήματα σε polygons των comm που δεν έχουν null zip code (αφοθ αυτές οι περιοχές είναι μη κατοικήσιμες)
#έπειτα βρίσκουμε τον ετήσιο Μ.Ο. για την συγκεκριμένη διετία και υπολογίζουμε τον δείκτη της ετήσιας μέσης αναλογίας εγκλημάτων ανά άτομο για κάθε περιοχή
# Create sedona context
sedona = SedonaContext.create(spark)
# Read the file from s3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Census_Blocks_2020.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")
# Formatting magic
flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

#Για κάθε διαφορετικό comm βρίσκουμε το συνολο των νοικοκυριων, των κατοικων και των εγκλημάτων που έχουν διαπραχθει εκει για το έτος 2020 και 2021
#υπολογίζουμε το ετήσιο κατακεφαλήν εισόδημα για κάθε περιοχή : Median houshold income * total housing / total population 
#το median houshold income το αποκτάμε : δεδομένο comm -> zip code -> median houshold income
zcta_agg_df = flattened_df \
    .filter(F.col("ZCTA20").isNotNull()) \
    .groupBy("COMM") \
    .agg(
        F.sum("POP20").alias("total_population"),
        F.sum("HOUSING20").alias("total_housing"),
        F.first("ZCTA20").alias("ZCTA20")
    ) \
    .orderBy("ZCTA20")

#zcta_agg_df.show(200)

#φορτώνουμε τώρα το dataset με το εισόδημα και κανουμε join με το zcta_agg_df για να αποκτήσουμε και την πληροφορία του ετήσιου μεσου εισοδήματος ανα νοικοκυριό
zip_income_schema = StructType([
    StructField("zip_code", IntegerType()),
    StructField("community", StringType()),
    StructField("Median Household Income in USD", StringType()) ] )

household_zip_income_df = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_income_2021.csv",
    header=True, schema=zip_income_schema, sep=";" 
)
#Το Dataset αυτό έχει το εισόδημα ως string εμεις αφού θελουμε να εκτελέσουυμε μαθηματικές πράξεις πρεπει να το καθαρίσουμε απο τα σύμβολα και μετα το ατοφιο νουμερο-string να το κανουμε cast σε int
clean_df = household_zip_income_df.withColumn(
    "income",
    F.regexp_replace(
        F.regexp_replace(
            F.col("Median Household Income in USD"), "\\$", ""
        ),
        ",",
        ""
    ).cast("int")
)

#clean_df.show(truncate = False)
#Θελουμε τωρα να ενωσουμε τα δυο dataset πανω στο zip code , αλλα πρώτα πρεπει να τα μετατρέψουμε σε ίδιο τυπο μεταβλητης και να ονομασουμε τις στηλες με το ίδιο όνομα. αυτές οι αλλαγές θα γινουν στο ZCTA20 δηλαδη στο censun block 

zcta_agg_df = zcta_agg_df.withColumn("ZCTA20", F.col("ZCTA20").cast("int"))
zcta_agg_df = zcta_agg_df.withColumnRenamed("ZCTA20", "zip_code")

comm_income_df = zcta_agg_df.join(clean_df, on="zip_code", how="inner")
comm_income_df = comm_income_df.orderBy(col("zip_code"))
#comm_income_df.show(truncate = False)

#τωρα παμε να υπολογίσουμε το κατα κεφαλην εισόδημα για κάθε περιοχή
income_df = comm_income_df.withColumn("per capita income", col("income") * col("total_housing") / col("total_population"))
#income_df.show(truncate = False)

#Φορτώνουμε το dataset των εγκληματων και φιλτραρουμε αυτα που εγιναν στην διετία 2020-2021
crimes_schema = StructType([
    StructField("dr_no", StringType()),
    StructField("date_rptd", StringType()),
    StructField("date_occ", StringType()),
    StructField("time_occ", StringType()),
    StructField("area", StringType()),
    StructField("area_name", StringType()),
    StructField("rpt_dist_no", StringType()),
    StructField("part_1_2", IntegerType()),
    StructField("crm_cd", StringType()),
    StructField("crm_cd_desc", StringType()),
    StructField("mocodes", StringType()),
    StructField("vict_age", StringType()),
    StructField("vict_sex", StringType()),
    StructField("vict_descent", StringType()),
    StructField("premis_cd", StringType()),
    StructField("premis_desc", StringType()),
    StructField("weapon_used_cd", StringType()),
    StructField("weapon_desc", StringType()),
    StructField("status", StringType()),
    StructField("status_desc", StringType()),
    StructField("crm_cd_1",StringType()),
    StructField("crm_cd_2",StringType()),
    StructField("crm_cd_3",StringType()),
    StructField("crm_cd_4",StringType()),
    StructField("location", StringType()),
    StructField("cross_street", StringType()),
    StructField("lat", FloatType()),
    StructField("lon", FloatType()),
])


crimes_2020_2025_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv", \
                                     header=False, \
                                     schema=crimes_schema)
years_df = crimes_2020_2025_df.withColumn("year", F.substring("date_occ", 1, 4).cast("int"))

crimes_df = years_df.filter((col("year") == 2020 ) | (col("year") == 2021))
crimes_df = crimes_df.select(col("year"), col("lat"), col("lon"))
#τωρα πρεπει να βάλουμε τα εγκλήματα σε περιοχές polygon. πρωτα πρεπει να εχουμε ενα dataset με αυτη την πληορορια (το flattened με ο,τι χρειάζομαι)
polygons_df = flattened_df \
    .filter(F.col("ZCTA20").isNotNull()) \
    .select(
        "COMM",
        "ZCTA20",
        "POP20",
        "HOUSING20",
        "geometry"
    )

#polygons_df.printSchema()
#polygons_df.show(5, truncate=False)

#παμε τωρα να μετατρεψουμε τα εγκληματα σε sedona points
crimes_points_df = crimes_df \
    .filter(col("lon").isNotNull() & col("lat").isNotNull()) \
    .withColumn(
        "geom_point",
        ST_Point(col("lon"), col("lat"))
    )

#και να ενταξουμε τα εγκληματα μεσα στα polygons των communities
crimes_to_polygons_df = crimes_points_df.join(
    polygons_df,
    ST_Within(col("geom_point"), col("geometry"))
)

crimes_per_comm_df = crimes_to_polygons_df.groupBy("COMM").count() \
    .withColumnRenamed("count", "crime_count")

#κανουμε join το income me to crimes per comm πανω στο comm για να εχουμε λη τη πληροφορια που χρειαζομαστε
final_df = crimes_per_comm_df.join(income_df, on="COMM", how="inner")
#final_df.show(truncate = False)
final_df = final_df.withColumn("average annual crimes", col("crime_count")/2)
final_df.show(truncate = False)
final_df.explain(True)

top_10_richest_df = final_df.orderBy(col("per capita income").desc()).limit(10)
top_10_richest_df.show(truncate = False)

top_10_poorest_df = final_df.orderBy(col("per capita income")).limit(10)
top_10_poorest_df.show(truncate = False)

corr_value_1 = final_df.stat.corr("average annual crimes", "per capita income")
corr_value_2 = top_10_richest_df.stat.corr("average annual crimes", "per capita income")
corr_value_3 = top_10_poorest_df.stat.corr("average annual crimes", "per capita income")
end_time = time.time()

print("Execution time for Query 5 (2 core, 4 executors, 4GB memory ): {:.4f} sec".format(end_time - start_time))
print(f"The correlation between average annual crimes and per capita income, for the entire LA area, is: {corr_value_1}")
print(f"The correlation between average annual crimes and per capita income for the 10 LA areas with the highest per capita income is: {corr_value_2}")
print(f"The correlation between average annual crimes and per capita income for the 10 LA areas with the lowest per capita income is: {corr_value_3}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----------+--------+----------------+-------------+-------------------------------------------------------------------------------------+------------------------------+------+------------------+---------------------+
|COMM                   |crime_count|zip_code|total_population|total_housing|community                                                                            |Median Household Income in USD|income|per capita income |average annual crimes|
+-----------------------+-----------+--------+----------------+-------------+-------------------------------------------------------------------------------------+------------------------------+------+------------------+---------------------+
|                       |1005       |91340   |5092953         |1781618      |San Fernando, Los Angeles (Mission Hills, Pacoima)                                   |$67,638                       |67638 |23661.140851682707|502.5                |
|Adams-Normandie        |793